# Creating ASN File for splittag Products

In [ ]:
from costools import timefilter
import calcos 

from astropy.io import fits                                            
from astropy.table import Table

import numpy as np
import glob
import os


import matplotlib.pyplot as plt
from matplotlib import gridspec

### timefilter example from https://costools.readthedocs.io/en/latest/timefilter.html

In [ ]:
!rm ./filtered_corrtag_*.fits
!rm filtered_asn.fits

data_dir = '../DayNight/data/'
output_dir = './output/'
plots_dir = output_dir + 'plots/'
!mkdir ./output
!mkdir ./output/plots/

filename_root = 'lb3q01meq'


timefilter.TimelineFilter(data_dir+filename_root+"_corrtag_a.fits", "filtered_corrtag_a.fits",  # your input names will be different
                              "sun_alt > 0.")
timefilter.TimelineFilter(data_dir+filename_root+"_corrtag_b.fits", "filtered_corrtag_b.fits",
                              "sun_alt > 0.")
# This must be repeated for all exposures

### Creating an ASN file out of the above files

In [ ]:
# this properly assigns the type of exposure in a way that CalCOS will recognize
type_dict = {'WAVECAL' : 'EXP-AWAVE',
             'EXTERNAL/SCI' : 'EXP-FP'}

files = glob.glob(data_dir+filename_root+"_corrtag_a.fits") # This will be different depending on your output name ## only need the A segment in the ASN, and calcos will find B for you

for f in files:
    
    # Adding the file details to the association table
    rootnames = [f]  # MEMNAME
    types = [type_dict[fits.getval(f, 'EXPTYPE')]] # MEMTYPE
    included = [True] # MEMPRSNT
    
    # Adding the ASN details to the end of the association table
    # the rootname needs to be the full name, not just the rootname
    asn_root = os.path.basename(f.split('corrtag')[0][:-1]) # removing the extra "_" at the end of this
    rootnames.append(asn_root.upper()) # MEMNAME
    types.append('PROD-FP') # MEMTYPE
    included.append(True) # MEMPRSNT
    
    # Putting together the fits table
    #   40 is the number of characters allowed in this field. If your rootname is longer than 40, 
    #     you will need to increase this
    c1 = fits.Column(name='MEMNAME', array=np.array(rootnames), format='40A') 
    c2 = fits.Column(name='MEMTYPE', array=np.array(types), format='14A')
    c3 = fits.Column(name='MEMPRSNT', format='L', array=included)
    t = fits.BinTableHDU.from_columns([c1, c2, c3])
    
    # Writing the fits table
    t.writeto(asn_root.lower()+'_asn.fits')

    print('Saved: {}'.format(asn_root.lower()+'_asn.fits'))


In [ ]:
# this properly assigns the type of exposure in a way that CalCOS will recognize
type_dict = {'WAVECAL' : 'EXP-AWAVE',
             'EXTERNAL/SCI' : 'EXP-FP'}

files = glob.glob('filtered_corrtag_a*') ## This will be different depending on your output name ## only need the A segment in the ASN, and calcos will find B for you

for f in files:
    
    # Adding the file details to the association table
    rootnames = [f]  # MEMNAME
    types = [type_dict[fits.getval(f, 'EXPTYPE')]] # MEMTYPE
    included = [True] # MEMPRSNT
    
    # Adding the ASN details to the end of the association table
    # the rootname needs to be the full name, not just the rootname
    asn_root = os.path.basename(f.split('corrtag')[0][:-1]) # removing the extra "_" at the end of this
    rootnames.append(asn_root.upper()) # MEMNAME
    types.append('PROD-FP') # MEMTYPE
    included.append(True) # MEMPRSNT
    
    # Putting together the fits table
    #   40 is the number of characters allowed in this field. If your rootname is longer than 40, 
    #     you will need to increase this
    c1 = fits.Column(name='MEMNAME', array=np.array(rootnames), format='40A') 
    c2 = fits.Column(name='MEMTYPE', array=np.array(types), format='14A')
    c3 = fits.Column(name='MEMPRSNT', format='L', array=included)
    t = fits.BinTableHDU.from_columns([c1, c2, c3])
    
    # Writing the fits table
    t.writeto(asn_root.lower()+'_asn.fits')

    print('Saved: {}'.format(asn_root.lower()+'_asn.fits'))


### Then run CalCOS on those individual files

In [ ]:
#First run with the initial "FULL" data with all time-tag datapoints
calcos.calcos('./lb3q01meq_asn.fits', outdir="full_data_outs", verbosity = 0)

In [ ]:
#Now run with the "FILTERED" data with only time-tag datapoints allowed by the filter
calcos.calcos('./filtered_asn.fits', outdir="filtered_data_outs", verbosity= 0)

In [ ]:
ftab = Table.read("./full_data_outs/"+filename_root+"_x1dsum.fits")['WAVELENGTH','FLUX']
ttab = Table.read("./filtered_data_outs/filtered_x1dsum.fits")['WAVELENGTH','FLUX']

combo_dict_f = {'WAVELENGTH':[], 'FLUX':[]}
combo_dict_t = {'WAVELENGTH':[], 'FLUX':[]}

for row in ftab:
    for key in row.colnames:
        combo_dict_f[key]+=(list(row[key]))
        
for row in ttab:
    for key in row.colnames:
        combo_dict_t[key]+=(list(row[key]))

In [ ]:
fig = plt.figure(figsize=(15, 15)) 
gs = gridspec.GridSpec(3,1, height_ratios=[1, 3,3]) 
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
ax2 = plt.subplot(gs[2])


ax0.scatter(combo_dict_f["WAVELENGTH"], combo_dict_f["FLUX"], s = 2, alpha = 1, c = 'C1',label = "Unfiltered")
ax0.scatter(combo_dict_t["WAVELENGTH"], combo_dict_t["FLUX"], s = 2, alpha = 1, c = 'C2', label = "Filtered to SUN_ALT<0")

ax1.scatter(combo_dict_f["WAVELENGTH"], combo_dict_f["FLUX"], s = 2, alpha = 1, c = 'C1',label = "Unfiltered")
ax0.set_xticks([])
ax2.scatter(combo_dict_t["WAVELENGTH"], combo_dict_t["FLUX"], s = 2, alpha = 1, c = 'C2', label = "Filtered to SUN_ALT<0")
ax1.set_xticks([])

ax0.legend(fontsize = 20)
ax1.legend(fontsize = 20)
ax2.legend(fontsize = 20)

ax2.set_xlabel("Wavelength [$\AA$]", fontsize = 30)
ax0.set_title("Spectra of Filtered and Unfiltered by `SUN_ALT`", size = 35)
ax1.set_ylabel("Flux [$\AA$]", fontsize = 30, y=0.2, horizontalalignment='center')
plt.tight_layout()

plt.savefig(plots_dir + 'compare_spectra_sunalt.png', dpi = 300)

In [ ]:
tab

In [ ]:
fits.info("../DayNight/lb3q01meq_corrtag_b.fits")
fits.info("./temp_corrtag_b.fits")

for extnum in [2]:
    print(Table.read("../DayNight/lb3q01meq_corrtag_b.fits", hdu = extnum),"\n=================================\n")

for extnum in [2,4]:
    print(Table.read("./temp_corrtag_b.fits", hdu = extnum))

In [ ]:
orig3= Table.read("../DayNight/lb3q01meq_corrtag_a.fits", hdu =3)
t,sa = orig3["TIME"],orig3["SUN_ALT"]
Y = np.polyval(np.polyfit(t,sa,1),t)
################
# plot it
fig = plt.figure(figsize=(12, 10)) 
gs = gridspec.GridSpec(2,1, height_ratios=[2, 1]) 
ax0 = plt.subplot(gs[0])
ax0.plot(t, sa)
ax0.set_xticks([])
ax0.set_ylabel("Sun Altitude", size = 14)

ax1 = plt.subplot(gs[1])
ax1.plot(t, sa - Y, "C1")
ax1.set_ylabel("Residual (Data - Model)", size = 14)
ax1.set_xlabel("Time (seconds)" , size = 20)

plt.tight_layout()

In [ ]:
np.polyval(np.polyfit(t,sa,1),249.98399353027344)

In [ ]:
Table.read("filtered_asn.fits")

First, exploring can we do this with rawtag, corrtag:

In [ ]:
from costools import timefilter

import calcos 

from astropy.io import fits      
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from astroquery.mast import Observations

In [ ]:
data_dir = './data/'
output_dir = './output/'

In [ ]:
pl = Observations.get_product_list(Observations.query_criteria(obs_id = 'lb3q01070'))
mask_ = np.where((pl["productSubGroupDescription"] == "CORRTAG_A") | (pl["productSubGroupDescription"] == "CORRTAG_B"))
Observations.download_products(pl[mask_], data_dir, cache=False)

In [ ]:
!rm output/temp*_corrtag_a.fits

timefilter.TimelineFilter(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits", output_dir+"temp2_corrtag_a.fits", # your input names will be different
                              "sun_alt > -40", verbose = True)

In [ ]:
orig1 = Table.read(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits", hdu =1)
temp1 = Table.read(output_dir+"temp2_corrtag_a.fits", hdu =1)

orig2 = Table.read(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits", hdu =2)
temp2 = Table.read(output_dir+"temp2_corrtag_a.fits", hdu =2)

orig3 = Table.read(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits", hdu =3)
temp3 = Table.read(output_dir+"temp2_corrtag_a.fits", hdu =3)

# orig0 = Table.read(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits", hdu =0)
temp0 = Table.read(output_dir+"temp2_corrtag_a.fits", hdu =4)

len(orig1),len(temp1)

In [ ]:
bins_ = np.arange(0,16)
xticks_ = [0] + list(np.linspace(start = 0, stop = 14 ,num = 15, dtype = int))
xtick_labels_ = [0] + list(np.logspace(start = 0, stop = 14, base=2, num = 15, dtype = int))
meanings = ['No Anomalies', 'Reed-Solomon Error', 'Hot Spot', 'Detector Shadow', 'Poorly Calibrated Region', # What each power of 2 means in order
            'Very-low Response Region', 'Background Feature', 'Burst', 'Pixel Out-of-bounds', 'Fill Data', 
            'Pulse Height Out-of-bounds', 'Low Response Region', 'Bad Time Interval', 'Low PHA Feature', 
           'Gain-sag Hole', 'Not Used']
xtick_labels_ = [str(bt)+":"+ mn for bt, mn in zip(xtick_labels_, meanings)]

np.warnings.filterwarnings('ignore')

fig, (ax0,ax1) = plt.subplots(2,1,figsize = (10,8))
ax0.hist(np.nan_to_num(np.log2(orig1["DQ"]), neginf=0), bins = bins_, alpha = 1, 
         color = 'C0', label = "Before Time Filter")
ax1.hist(np.nan_to_num(np.log2(temp1["DQ"]), neginf=0),bins = bins_, alpha = 1, 
         color = 'C1', label = "After Time Filter")
ax0.set_xticks([])
ax1.set_xticks(np.add(0.5,xticks_))
ax1.set_xticklabels(xtick_labels_, rotation = 90, fontsize = 14)
ax0.legend()
ax1.legend()
ax0.set_title("DQ Flags Before/After of Sun-Altitude Filter", size = 20)
plt.tight_layout()

In [ ]:
fits.getheader(data_dir+ "mastDownload/HST/lb3q01kbq/" +"lb3q01kbq_corrtag_a.fits",ext=1)["SDQFLAGS"],fits.getheader(output_dir+ "temp2_corrtag_a.fits",ext=1)["SDQFLAGS"]

In [ ]:
def interpret_sdq(flags):
    print("Bit\tFlagged?\tMeaning")
    for i, char in enumerate(bin(flags)[::-1][:-2]):
        print(2**(i),"\t", bool(int(char)),"\t", meanings[i+1])

interpret_sdq(8346)

In [ ]:
orig2

In [ ]:
temp3

In [ ]:
temp0

In [ ]:
Table.read(output_dir+"temp2_corrtag_a.fits", hdu =4)

In [ ]:
fits.info(output_dir+ "temp2_corrtag_a.fits")

In [ ]:
fits.info(data_dir + 'lbgu17qnq_corrtag_a.fits')

In [ ]:
glob.glob(data_dir + "**/*fits", recursive=True)

In [ ]:
t,sa = orig3["TIME"],orig3["SUN_ALT"]
plt.scatter(t,sa, s =.1)